In [107]:
import requests
import json

from lxml import html

import matplotlib.pyplot as plt
import numpy as np

import random
import time
from pprint import pprint

In [108]:
YELP_TOKEN = "26oHkEyYL4zBMAzhjcSqhWF-bd0H_esyiB5L_1koKyCVGIAusr72qmVgjctQClrDbjpH5gwXP7aC1Ene5qKraq-cYGq9xJ3ySoj4PTpd8dvSVZq6BJlmuDyTV2sCXnYx"
user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
headers = {'User-Agent': user_agent}

## Scraping parameters
SLEEP_TIME = 0.2 #200ms
MAX_BUSINESSES = 50
REVIEW_LOCATION = 'Toronto'


#r = requests.get("https://pokeapi.co/api/v2/pokemon-species/")
#r = requests.get("https://www.yelp.com/biz/mZRKH9ngRY92bI_irrHq6w/review_feed?rl=en&sort_by=rating_asc&q=&start=0")
#r

In [109]:
r = requests.get("https://api.yelp.com/v3/businesses/search?location=%s&limit=%s" % (REVIEW_LOCATION, str(MAX_BUSINESSES)), headers={"Authorization": "Bearer %s" % YELP_TOKEN})

In [110]:
def get_business_review(business_id):

    best_reviews_url_suffix = '/review_feed?rl=en&sort_by=rating_desc&q=&start='
    worst_reviews_url_suffix = '/review_feed?rl=en&sort_by=rating_asc&q=&start='
    
    review_suffixes=[]
    review_suffixes.append(best_reviews_url_suffix)
    review_suffixes.append(worst_reviews_url_suffix)
    
    page_num = 0 #can change this to 
    review_labels = []
    
    for review_suffix in review_suffixes:
        review_url = 'https://www.yelp.com/biz/'+business_id+review_suffix+str(page_num) #the 'start' number here cab be changed for pagination
        page = requests.get(review_url, headers = headers)
        print(review_url,page.status_code, page.reason)
        #print(page.json().keys()) #Outputs: dict_keys(['snippet_serial', 'header', 'pagination', 'review_list', 'full_page_url'])
                                  #Can use pagination to extract maximum number of pages to iterate through if we wanted more reviews

        #for page_num in range(0,maxPage) OR might extract the href from the pagination html and use that directly
                  
        tree = html.fromstring(page.json()['review_list'])
    
        review_boxes = '//div[@class="review review--with-sidebar"]'
        reviews = tree.xpath(review_boxes)
    
        for review in reviews:
            xpath_rating = './/div[contains(@class, "i-stars i-stars--regular")]/@title'
            xpath_user  = './/a[@id="dropdown_user-name"]//text()'
            xpath_body    = './/p[@lang="en"]//text()'
        
            rating_str=review.xpath(xpath_rating) #returns a list. Can have more than one rating in the case of updated reviews. The first rating is the most recent
            rating_val = rating_str[0][0]
            rating = int(rating_val)
            user  = review.xpath(xpath_user)
            body_sentences = review.xpath(xpath_body)
            body = ''
            for sentence in body_sentences:
                body=body+" "+sentence
            #print(user)
            #print(body)
            #print(rating)
            #print()
        
            review_labels.append([body,"positive" if rating>3 else "negative"])
        
    return review_labels

In [111]:
bus_ids = []
for business in r.json()['businesses']:
    bus_ids.append((business['id'],business['url']))
    #reviews = requests.get("https://api.yelp.com/v3/businesses/%s/reviews" % business['id'], headers={"Authorization": "Bearer %s" % YELP_TOKEN})
    #print(len(reviews.json()['reviews']))
    
print(len(bus_ids))
print(reviews)

50
<Response [200]>


In [112]:
#page = get_business_review(bus_ids[0][0])
review_labels=[]
for (x,y) in bus_ids:
    review_label_business = get_business_review(x)
    time.sleep(SLEEP_TIME)
    review_labels += review_label_business

https://www.yelp.com/biz/r_BrIgzYcwo1NAuG9dLbpg/review_feed?rl=en&sort_by=rating_desc&q=&start=0 200 OK
https://www.yelp.com/biz/r_BrIgzYcwo1NAuG9dLbpg/review_feed?rl=en&sort_by=rating_asc&q=&start=0 200 OK
https://www.yelp.com/biz/iGEvDk6hsizigmXhDKs2Vg/review_feed?rl=en&sort_by=rating_desc&q=&start=0 200 OK
https://www.yelp.com/biz/iGEvDk6hsizigmXhDKs2Vg/review_feed?rl=en&sort_by=rating_asc&q=&start=0 200 OK
https://www.yelp.com/biz/RtUvSWO_UZ8V3Wpj0n077w/review_feed?rl=en&sort_by=rating_desc&q=&start=0 200 OK
https://www.yelp.com/biz/RtUvSWO_UZ8V3Wpj0n077w/review_feed?rl=en&sort_by=rating_asc&q=&start=0 200 OK
https://www.yelp.com/biz/N93EYZy9R0sdlEvubu94ig/review_feed?rl=en&sort_by=rating_desc&q=&start=0 200 OK
https://www.yelp.com/biz/N93EYZy9R0sdlEvubu94ig/review_feed?rl=en&sort_by=rating_asc&q=&start=0 200 OK
https://www.yelp.com/biz/mZRKH9ngRY92bI_irrHq6w/review_feed?rl=en&sort_by=rating_desc&q=&start=0 200 OK
https://www.yelp.com/biz/mZRKH9ngRY92bI_irrHq6w/review_feed?rl=en&so

https://www.yelp.com/biz/c35qzLN6ItDpVIoj2uQW4Q/review_feed?rl=en&sort_by=rating_desc&q=&start=0 200 OK
https://www.yelp.com/biz/c35qzLN6ItDpVIoj2uQW4Q/review_feed?rl=en&sort_by=rating_asc&q=&start=0 200 OK
https://www.yelp.com/biz/eD-XLSo-j8uIoTnIlrG8YA/review_feed?rl=en&sort_by=rating_desc&q=&start=0 200 OK
https://www.yelp.com/biz/eD-XLSo-j8uIoTnIlrG8YA/review_feed?rl=en&sort_by=rating_asc&q=&start=0 200 OK
https://www.yelp.com/biz/lDpVyjENOz_qbxbRoyBi8w/review_feed?rl=en&sort_by=rating_desc&q=&start=0 200 OK
https://www.yelp.com/biz/lDpVyjENOz_qbxbRoyBi8w/review_feed?rl=en&sort_by=rating_asc&q=&start=0 200 OK
https://www.yelp.com/biz/L-Yj3Y1bYKTSg4uE9KTVBw/review_feed?rl=en&sort_by=rating_desc&q=&start=0 200 OK
https://www.yelp.com/biz/L-Yj3Y1bYKTSg4uE9KTVBw/review_feed?rl=en&sort_by=rating_asc&q=&start=0 200 OK
https://www.yelp.com/biz/4_EgrMY-EI-i-xyWixI2qg/review_feed?rl=en&sort_by=rating_desc&q=&start=0 200 OK
https://www.yelp.com/biz/4_EgrMY-EI-i-xyWixI2qg/review_feed?rl=en&so

In [113]:
#review_labels

In [114]:
f = open("data.json", "w")
f.write(str(review_labels))
#f.write('\n'.join('%s %s' % x for x in review_labels))
#for review_label in review_labels:
#    f.write(review_label)
#    f.write(',')

#f.rstrip(',')
#f.write(']')
f.close()

len(review_labels)

2000

In [115]:
#Extract features
review_features = []
review_features2 = []
for (body,rating) in review_labels:
    feature_list = []
    for word in body.split(" "):
            feature_list.append(word)
    review_features.append((feature_list, rating ))
#review_features

In [116]:
len(review_features)

2000

In [117]:
from nltk.sentiment import SentimentAnalyzer
import nltk.sentiment.util
from nltk.classify import NaiveBayesClassifier

TRAIN_SAMPLE_COUNT = int(len(review_features)*0.7)
random.shuffle(review_features)
training_docs = review_features[:TRAIN_SAMPLE_COUNT]
test_docs = review_features[TRAIN_SAMPLE_COUNT:]

print("Training: %d, Testing: %d" % (len(training_docs), len(test_docs)))

sentim_analyzer = SentimentAnalyzer()

Training: 1400, Testing: 600


In [118]:
all_words_neg = sentim_analyzer.all_words([nltk.sentiment.util.mark_negation(doc) for doc in training_docs])
#all_words_neg

In [119]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
sentim_analyzer.add_feat_extractor(nltk.sentiment.util.extract_unigram_feats, unigrams=unigram_feats)

In [120]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(test_docs)

In [121]:
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
     print('{0}: {1}'.format(key, value))

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.74
F-measure [negative]: 0.7291666666666666
F-measure [positive]: 0.75
Precision [negative]: 0.8015267175572519
Precision [positive]: 0.6923076923076923
Recall [negative]: 0.6687898089171974
Recall [positive]: 0.8181818181818182
